# TODO: Title

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO's and use more than one TODO code cell to do all your tasks.

In [1]:
# TODO: Install any packages that you might need
# For instance, you will need the smdebug package
!pip install smdebug

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [2]:
# TODO: Import any packages that you might need
# For instance you will need Boto3 and Sagemaker
import sagemaker
import boto3
sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/DEMO-pytorch-cifar"

role = sagemaker.get_execution_role()

## Dataset
TODO: Explain what dataset you are using for this project. Maybe even give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understand of it.

In [25]:
from torchvision.datasets import CIFAR10
from torchvision import transforms

## Download from pytorch to local directory
local_dir = 'data'
CIFAR10.mirrors = ["https://sagemaker-sample-files.s3.amazonaws.com/datasets/image/CIFAR10/"]
CIFAR10(
    local_dir,
    download=True,
    transform=transforms.Compose(
        [transforms.ToTensor()]
    )
)

0it [00:00, ?it/s]

Extracting data/cifar-10-python.tar.gz to data


Dataset CIFAR10
    Number of datapoints: 50000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
           )

In [3]:
#TODO: Fetch and upload the data to AWS S3
inputs = sagemaker_session.upload_data(path="data", bucket=bucket, key_prefix=prefix)
print("input spec (in this case, just an S3 path): {}".format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-us-east-1-990956898706/sagemaker/DEMO-pytorch-cifar


## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

The chosen hyper parameters are the learning rate which is the most important parameter one can spend time tuning because it would help approach minimum error quickly if it's chosen correctly.
The other hyperparameter chosen is the number of epochs, It's important to know whether higher number of epochs helps the network more in cnvergence or not.

In [76]:
 #TODO: Initialise your hyperparameters
hyperparameter_ranges = {
    "lr": ContinuousParameter(0.001, 0.1),
    "epochs": CategoricalParameter([5, 10, 15, 20]),
}

In [89]:
#TODO: Create estimators for your HPs

 # TODO: Your estimator here
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point="hpo.py",
    role=role,
    py_version='py36',
    framework_version="1.8",
    instance_count=1,
    instance_type="ml.m5.large"
)


In [90]:
objective_metric_name = "average test loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "average test loss", "Regex": "Test set: Average loss: ([0-9\\.]+)"}]



In [91]:
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)


tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=4,
    max_parallel_jobs=4,
    objective_type=objective_type,
)# TODO: Your HP tuner here

Don't worry about the error below, I just stopped the notebook but the tuner was created successfully and stored in the tuner variable.

In [93]:
# TODO: Fit your HP Tuner
tuner.fit({"training":inputs}) # TODO: Remember to include your data channels

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

KeyboardInterrupt: 

In [95]:
# TODO: Get the best estimators and the best HPs
best_estimator = tuner.best_estimator() 


2022-11-30 08:00:46 Starting - Preparing the instances for training
2022-11-30 08:00:46 Downloading - Downloading input data
2022-11-30 08:00:46 Training - Training image download completed. Training in progress.
2022-11-30 08:00:46 Uploading - Uploading generated training model
2022-11-30 08:00:46 Completed - Resource released due to keep alive period expiry


In [96]:
#Get the hyperparameters of the best trained model
best_estimator.hyperparameters()

{'_tuning_objective_metric': '"average test loss"',
 'epochs': '"10"',
 'lr': '0.002520061454874928',
 'sagemaker_container_log_level': '20',
 'sagemaker_estimator_class_name': '"PyTorch"',
 'sagemaker_estimator_module': '"sagemaker.pytorch.estimator"',
 'sagemaker_job_name': '"pytorch-training-2022-11-30-06-50-24-428"',
 'sagemaker_program': '"hpo.py"',
 'sagemaker_region': '"us-east-1"',
 'sagemaker_submit_directory': '"s3://sagemaker-us-east-1-990956898706/pytorch-training-2022-11-30-06-50-24-428/source/sourcedir.tar.gz"'}

It was Found that Number of Epochs=10 and learning rate of 0.0025 training job has the lowest test loss. That's why they will be used in the next section

## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [29]:
# TODO# install dependencies
# install dependencies
!pip install smdebug

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [4]:
#Set up debugging and profiling rules and hooks
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role

from sagemaker.debugger import (
    Rule,
    DebuggerHookConfig,
    rule_configs,
)

from sagemaker.debugger import Rule, ProfilerRule, rule_configs

rules = [
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.poor_weight_initialization()),
    Rule.sagemaker(rule_configs.loss_not_decreasing()),
    ProfilerRule.sagemaker(rule_configs.LowGPUUtilization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
]

In [5]:
from sagemaker.debugger import ProfilerConfig, FrameworkProfile

hook_config = DebuggerHookConfig(
    
    hook_parameters = {"train.save_interval": "100", "eval.save_interval": "10"}
)

profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500, framework_profile_params=FrameworkProfile(num_steps=10)
)


In [13]:
hyperparameters = {
    "BatchSize": 32, #From Previous Task
    "epochs": 10, #From previous tuner
    "lr":  0.0025 #From Previous Tuner
}



In [14]:
# TODO: Create and fit an estimator

# TODO: Your estimator here

#TODO: Create the estimator to train your model
import sagemaker
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
## The line of code to be executed will be something like that
#  pytorch_cifar.py --batch-size 32 --lr 0.001 --test-batch-size 100
estimator = PyTorch(
    entry_point="train_model.py",
    base_job_name="sagemaker-script-mode",
    role=get_execution_role(),
    instance_count=1,
    instance_type="ml.m5.xlarge",
    hyperparameters = hyperparameters,
    framework_version="1.8",
    py_version="py36",
    ## Debugger parameters
    rules=rules,
    debugger_hook_config=hook_config,
    ## Profiler parameters
    profiler_config=profiler_config,
)


In [ ]:
estimator.fit({"training":inputs})


2022-12-01 17:54:33 Starting - Starting the training job...
2022-12-01 17:55:01 Starting - Preparing the instances for trainingVanishingGradient: InProgress
Overfit: InProgress
Overtraining: InProgress
PoorWeightInitialization: InProgress
LossNotDecreasing: InProgress
LowGPUUtilization: InProgress
ProfilerReport: InProgress
......
2022-12-01 17:56:01 Downloading - Downloading input data...
2022-12-01 17:56:22 Training - Downloading the training image.....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-12-01 17:57:13,151 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-12-01 17:57:13,153 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-12-01 17:57:13,162 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-12-01 17:57:13,168 sagemaker_pytorch_container.training INFO     Invoking user training s


2022-12-01 17:57:22 Training - Training image download completed. Training in progress.[2022-12-01 17:57:38.200 algo-1:28 INFO python_profiler.py:182] Dumping cProfile stats to /opt/ml/output/profiler/framework/pytorch/cprofile/28-algo-1/train-1-stepstart-1669917439191224.0_train-1-forwardpassend-1669917458200272.0/python_stats.
[2022-12-01 17:57:38.280 algo-1:28 INFO python_profiler.py:182] Dumping cProfile stats to /opt/ml/output/profiler/framework/pytorch/cprofile/28-algo-1/train-1-forwardpassend-1669917458202590.5_train-2-stepstart-1669917458279725.5/python_stats.
[2022-12-01 17:57:56.836 algo-1:28 INFO python_profiler.py:182] Dumping cProfile stats to /opt/ml/output/profiler/framework/pytorch/cprofile/28-algo-1/train-2-stepstart-1669917458282554.0_train-2-forwardpassend-1669917476836090.5/python_stats.
[2022-12-01 17:57:56.906 algo-1:28 INFO python_profiler.py:182] Dumping cProfile stats to /opt/ml/output/profiler/framework/pytorch/cprofile/28-algo-1/train-2-forwardpassend-166991

Train Epoch: 4 [28800/50000 (58%)] Loss: 1.914499
Train Epoch: 4 [32000/50000 (64%)] Loss: 1.810145
Train Epoch: 4 [35200/50000 (70%)] Loss: 1.988090
Train Epoch: 4 [38400/50000 (77%)] Loss: 2.253108
Train Epoch: 4 [41600/50000 (83%)] Loss: 1.827440
Train Epoch: 4 [44800/50000 (90%)] Loss: 1.930237
Train Epoch: 4 [48000/50000 (96%)] Loss: 1.739476
Test set: Average loss: 0.0025, Accuracy: 3844/10000 (38%)
Train Epoch: 5 [3200/50000 (6%)] Loss: 1.545722
Train Epoch: 5 [6400/50000 (13%)] Loss: 1.777458
Train Epoch: 5 [9600/50000 (19%)] Loss: 1.688760
Train Epoch: 5 [12800/50000 (26%)] Loss: 1.505260
Train Epoch: 5 [16000/50000 (32%)] Loss: 1.673839
Train Epoch: 5 [19200/50000 (38%)] Loss: 1.623136
Train Epoch: 5 [22400/50000 (45%)] Loss: 1.541820
Train Epoch: 5 [25600/50000 (51%)] Loss: 1.621118
Train Epoch: 5 [28800/50000 (58%)] Loss: 1.661690
Train Epoch: 5 [32000/50000 (64%)] Loss: 2.209359
Train Epoch: 5 [35200/50000 (70%)] Loss: 1.860816
Train Epoch: 5 [38400/50000 (77%)] Loss: 1.63


2022-12-01 18:50:20 Completed - Training job completed
VanishingGradient: NoIssuesFound
Overfit: Error
Overtraining: IssuesFound
PoorWeightInitialization: Error
LossNotDecreasing: NoIssuesFound
LowGPUUtilization: NoIssuesFound
ProfilerReport: IssuesFound
Training seconds: 3253
Billable seconds: 3253


In [16]:
from smdebug.trials import create_trial
from smdebug.core.modes import ModeKeys

trial = create_trial(estimator.latest_job_debugger_artifacts_path())

[2022-12-01 19:20:07.104 ip-172-16-6-150.ec2.internal:9403 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-12-01 19:20:07.147 ip-172-16-6-150.ec2.internal:9403 INFO s3_trial.py:42] Loading trial debug-output at path s3://sagemaker-us-east-1-990956898706/sagemaker-script-mode-2022-12-01-17-54-32-629/debug-output


In [17]:
# TODO: Can you print the names of all the tensors that were tracked
print(trial.tensor_names())


[2022-12-01 19:20:09.727 ip-172-16-6-150.ec2.internal:9403 INFO trial.py:198] Training has ended, will refresh one final time in 1 sec.
[2022-12-01 19:20:10.748 ip-172-16-6-150.ec2.internal:9403 INFO trial.py:210] Loaded all steps
['CrossEntropyLoss_output_0', 'gradient/ResNet_fc.0.bias', 'gradient/ResNet_fc.0.weight', 'gradient/ResNet_fc.1.bias', 'gradient/ResNet_fc.1.weight', 'gradient/ResNet_fc.2.bias', 'gradient/ResNet_fc.2.weight', 'layer1.0.relu_input_0', 'layer1.0.relu_input_1', 'layer1.0.relu_input_2', 'layer1.1.relu_input_0', 'layer1.1.relu_input_1', 'layer1.1.relu_input_2', 'layer1.2.relu_input_0', 'layer1.2.relu_input_1', 'layer1.2.relu_input_2', 'layer2.0.relu_input_0', 'layer2.0.relu_input_1', 'layer2.0.relu_input_2', 'layer2.1.relu_input_0', 'layer2.1.relu_input_1', 'layer2.1.relu_input_2', 'layer2.2.relu_input_0', 'layer2.2.relu_input_1', 'layer2.2.relu_input_2', 'layer2.3.relu_input_0', 'layer2.3.relu_input_1', 'layer2.3.relu_input_2', 'layer3.0.relu_input_0', 'layer3.0

In [18]:
Loss = trial.tensor_names()[0]
print(Loss)

CrossEntropyLoss_output_0


In [19]:
# TODO: Can you print the number of datapoints for one of those tensors
# for both train and eval mode
print(len(trial.tensor(Loss).steps(mode=ModeKeys.TRAIN)))
print(len(trial.tensor(Loss).steps(mode=ModeKeys.EVAL)))

4
29


In [20]:
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob

tj = TrainingJob(estimator._current_job_name)
tj.wait_for_sys_profiling_data_to_be_available()

ProfilerConfig:{'S3OutputPath': 's3://sagemaker-us-east-1-990956898706/', 'ProfilingIntervalInMilliseconds': 500, 'ProfilingParameters': {'DataloaderProfilingConfig': '{"StartStep": 0, "NumSteps": 10, "MetricsRegex": ".*", }', 'DetailedProfilingConfig': '{"StartStep": 0, "NumSteps": 10, }', 'FileOpenFailThreshold': '50', 'HorovodProfilingConfig': '{"StartStep": 0, "NumSteps": 10, }', 'LocalPath': '/opt/ml/output/profiler', 'PythonProfilingConfig': '{"StartStep": 0, "NumSteps": 10, "ProfilerName": "cprofile", "cProfileTimer": "total_time", }', 'RotateFileCloseIntervalInSeconds': '60', 'RotateMaxFileSizeInBytes': '10485760', 'SMDataParallelProfilingConfig': '{"StartStep": 0, "NumSteps": 10, }'}}
s3 path:s3://sagemaker-us-east-1-990956898706/sagemaker-script-mode-2022-12-01-17-54-32-629/profiler-output


Profiler data from system is available


In [21]:
# TODO: Plot a debugging output.
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts

system_metrics_reader = tj.get_systems_metrics_reader()
system_metrics_reader.refresh_event_file_list()

view_timeline_charts = TimelineCharts(
    system_metrics_reader,
    framework_metrics_reader=None,
    select_dimensions=["CPU", "GPU"],
    select_events=["total"],
)

[2022-12-01 19:20:16.736 ip-172-16-6-150.ec2.internal:9403 INFO metrics_reader_base.py:134] Getting 55 event files
select events:['total']
select dimensions:['CPU', 'GPU']
filtered_events:{'total'}
filtered_dimensions:{'CPUUtilization-nodeid:algo-1'}


GPU chart isn't provided since GPU device wasn't enabled


In [22]:
rule_output_path = estimator.output_path + estimator.latest_training_job.job_name + "/rule-output"
print(f"You will find the profiler report in {rule_output_path}")

You will find the profiler report in s3://sagemaker-us-east-1-990956898706/sagemaker-script-mode-2022-12-01-17-54-32-629/rule-output


In [23]:
! aws s3 ls {rule_output_path} --recursive

2022-12-01 18:49:55     390776 sagemaker-script-mode-2022-12-01-17-54-32-629/rule-output/ProfilerReport/profiler-output/profiler-report.html
2022-12-01 18:49:55     241552 sagemaker-script-mode-2022-12-01-17-54-32-629/rule-output/ProfilerReport/profiler-output/profiler-report.ipynb
2022-12-01 18:49:49        192 sagemaker-script-mode-2022-12-01-17-54-32-629/rule-output/ProfilerReport/profiler-output/profiler-reports/BatchSize.json
2022-12-01 18:49:49        200 sagemaker-script-mode-2022-12-01-17-54-32-629/rule-output/ProfilerReport/profiler-output/profiler-reports/CPUBottleneck.json
2022-12-01 18:49:49       2088 sagemaker-script-mode-2022-12-01-17-54-32-629/rule-output/ProfilerReport/profiler-output/profiler-reports/Dataloader.json
2022-12-01 18:49:49        127 sagemaker-script-mode-2022-12-01-17-54-32-629/rule-output/ProfilerReport/profiler-output/profiler-reports/GPUMemoryIncrease.json
2022-12-01 18:49:49        199 sagemaker-script-mode-2022-12-01-17-54-32-629/rule-output/Profile

In [24]:
! aws s3 cp {rule_output_path} ./ --recursive #save them locally

download: s3://sagemaker-us-east-1-990956898706/sagemaker-script-mode-2022-12-01-17-54-32-629/rule-output/ProfilerReport/profiler-output/profiler-reports/BatchSize.json to ProfilerReport/profiler-output/profiler-reports/BatchSize.json
download: s3://sagemaker-us-east-1-990956898706/sagemaker-script-mode-2022-12-01-17-54-32-629/rule-output/ProfilerReport/profiler-output/profiler-reports/GPUMemoryIncrease.json to ProfilerReport/profiler-output/profiler-reports/GPUMemoryIncrease.json
download: s3://sagemaker-us-east-1-990956898706/sagemaker-script-mode-2022-12-01-17-54-32-629/rule-output/ProfilerReport/profiler-output/profiler-report.ipynb to ProfilerReport/profiler-output/profiler-report.ipynb
download: s3://sagemaker-us-east-1-990956898706/sagemaker-script-mode-2022-12-01-17-54-32-629/rule-output/ProfilerReport/profiler-output/profiler-reports/CPUBottleneck.json to ProfilerReport/profiler-output/profiler-reports/CPUBottleneck.json
download: s3://sagemaker-us-east-1-990956898706/sagemake

In [25]:
import os

# get the autogenerated folder name of profiler report
profiler_report_name = [
    rule["RuleConfigurationName"]
    for rule in estimator.latest_training_job.rule_job_summary()
    if "Profiler" in rule["RuleConfigurationName"]
][0]

In [26]:
profiler_report_name

'ProfilerReport'

In [27]:
import IPython

IPython.display.HTML(filename=profiler_report_name + "/profiler-output/profiler-report.html")

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

When Deploying the endpoint, It appears an error mentioning that smdebug is not defined, It seems that while deployment the instance won't access the smdebug library which is actually not important in the deployment phase so a protection against this error is providedin the train_model.py.
First while importing the module the following try except is added:

    try:
        import smdebug.pytorch as smd
        from smdebug.profiler.utils import str2bool
    except:
        pass
    
Second when initializing the hook and assigning a model, the following code is added:

    try:
        hook = smd.Hook.create_from_json_file()
        hook.register_hook(model)
    except:
        hook=0
Third, Training and Testing will receive the hook value, depending on which a protection was added inside their loops:

    ##Protection against Error Smdebug not defined
    if (hook==0):
        pass
    else:
        hook.set_mode(smd.modes.TRAIN)
    ############################################## 
    ##Protection against Error Smdebug not defined
    if (hook==0):
        pass
    else:
        hook.set_mode(smd.modes.EVAL)
    ############################################## 
Finally the train_model.py is working for both Training and Deployment.

## Model Deploying

In [28]:
# TODO: Deploy your model to an endpoint
# TODO: Add your deployment configuration like instance type and number of instances
#change the instance type to support msdebug
predictor=estimator.deploy(initial_instance_count=1, instance_type="ml.m5.xlarge")  

-----!

In [29]:
predictor.endpoint_name

'sagemaker-script-mode-2022-12-01-19-23-04-839'

## Prepare data for inference

In [30]:
# TODO: Your code to load and preprocess image to send to endpoint for prediction
import gzip 
import numpy as np
import random
import os

file = 'data/cifar-10-batches-py/data_batch_1'
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        Data = pickle.load(fo, encoding='bytes')
    return Data

Data_extracted=unpickle(file)
data=np.reshape(Data_extracted[b'data'][0], (3, 32, 32))

In [31]:
Imgs_train=unpickle('data/cifar-10-batches-py/data_batch_1') #image from the training dataset

The following datapoint is for a leptodactylus which is a sort  of a frog that takes label 6.

In [192]:
#Information about the picked Image
n=0
print(Imgs_train[b'labels'][n])
print(Imgs_train[b'data'][n].shape)
print(Imgs_train[b'filenames'][n])
Img_train=np.reshape(Imgs_train[b'data'][n], (3, 32, 32))
print(Img_train)

6
(3072,)
b'leptodactylus_pentadactylus_s_000004.png'
[[[ 59  43  50 ... 158 152 148]
  [ 16   0  18 ... 123 119 122]
  [ 25  16  49 ... 118 120 109]
  ...
  [208 201 198 ... 160  56  53]
  [180 173 186 ... 184  97  83]
  [177 168 179 ... 216 151 123]]

 [[ 62  46  48 ... 132 125 124]
  [ 20   0   8 ...  88  83  87]
  [ 24   7  27 ...  84  84  73]
  ...
  [170 153 161 ... 133  31  34]
  [139 123 144 ... 148  62  53]
  [144 129 142 ... 184 118  92]]

 [[ 63  45  43 ... 108 102 103]
  [ 20   0   0 ...  55  50  57]
  [ 21   0   8 ...  50  50  42]
  ...
  [ 96  34  26 ...  70   7  20]
  [ 96  42  30 ...  94  34  34]
  [116  94  87 ... 140  84  72]]]


In [193]:
data_batch_index=np.expand_dims(Img_train, axis=0)
data_batch_index.shape

(1, 3, 32, 32)


 Query the Endpoint

In [194]:
# TODO: Run an prediction on the endpoint
import torch
response = predictor.predict(torch.Tensor(data_batch_index)) #convert array to a byte tensor

# TODO: Query the endpoint
print(response)

[[-1916.40600586 -1659.82397461 -4214.36914062 -1532.24328613
  -2554.58276367 -5634.63916016  -360.0138855  -6080.62988281
  -1598.20654297  -540.77832031]]


In [195]:
m = torch.nn.Softmax(dim=1)

Note that the highest softmax of the element with index  6 that represents a frog

In [196]:
m(torch.Tensor(response))

tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]])

For another imahge from the training dataset "camion" which is a sort of trucks that has a label of 9.

In [197]:
#Information about the picked Image
n=1
print(Imgs_train[b'labels'][n])
print(Imgs_train[b'data'][n].shape)
print(Imgs_train[b'filenames'][n])
Img_train2=np.reshape(Imgs_train[b'data'][n], (3, 32, 32))
print(Img_train2)

9
(3072,)
b'camion_s_000148.png'
[[[154 126 105 ...  91  87  79]
  [140 145 125 ...  96  77  71]
  [140 139 115 ...  79  68  67]
  ...
  [175 156 154 ...  42  61  93]
  [165 156 159 ... 103 123 131]
  [163 158 163 ... 143 143 143]]

 [[177 137 104 ...  95  90  81]
  [160 153 125 ...  99  80  73]
  [155 146 115 ...  82  70  69]
  ...
  [167 154 160 ...  34  53  83]
  [154 152 161 ...  93 114 121]
  [148 148 156 ... 133 134 133]]

 [[187 136  95 ...  71  71  70]
  [169 154 118 ...  78  62  61]
  [164 149 112 ...  64  55  55]
  ...
  [166 160 170 ...  36  57  91]
  [128 130 142 ...  96 120 131]
  [120 122 133 ... 139 142 144]]]


In [198]:
data_batch_index2=np.expand_dims(Img_train2, axis=0)
data_batch_index2.shape

(1, 3, 32, 32)

In [199]:
# TODO: Run an prediction on the endpoint
import torch
response2 = predictor.predict(torch.Tensor(data_batch_index2)) #convert array to a byte tensor

# TODO: Query the endpoint
print(response2)

[[-1308.46118164   898.83898926 -5117.67919922 -1783.74914551
  -1160.02124023 -8379.05273438   187.16600037 -7510.72802734
    556.75323486  2095.21606445]]


In [200]:
m = torch.nn.Softmax(dim=1)

The output Indicates that element in index 9 has the highest value

In [201]:
m(torch.Tensor(response2))

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

### Cleanup

After you have finished with this exercise, remember to delete the prediction endpoint to release the instance associated with it

In [ ]:
predictor.delete_endpoint()


In [3]:
!zip -r CD0387-deep-learning-topics-within-computer-vision-nlp-project-starter.zip "CD0387-deep-learning-topics-within-computer-vision-nlp-project-starter"

	zip warning: name not matched: /CD0387-deep-learning-topics-within-computer-vision-nlp-project-starter

zip error: Nothing to do! (try: zip -r CD0387-deep-learning-topics-within-computer-vision-nlp-project-starter.zip . -i /CD0387-deep-learning-topics-within-computer-vision-nlp-project-starter)
